## Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
author: cdamien 2023


# BigQuery Performance test

Author: cdamien

This notebook can be used to help in the context of a query performance test and provide several useful scripts to check:
1. Check various aspects of the tables in a dataset
2. Check assigned slots
3. Perform a performance test of your queries (especially what would be the effect of a high number of concurrent queries)





## 1. Initialization
---

In [ ]:
from google.colab import files
from google.cloud import bigquery

# Initial setup
global PROJECT_NAME
PROJECT_NAME = "my-project"
global DATASET
DATASET = "my-dataset"
global REGION
REGION = "us-central1"

### Authentificate your user

In [ ]:
# use this if you want to authentificate with the current user
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
client = bigquery.Client(project=PROJECT_NAME)

In [ ]:
# use this if you want to upload and specify a local key file
from google.oauth2 import service_account
uploaded = files.upload()
credentials = service_account.Credentials.from_service_account_file("your-service-account.json") #update filename here
client = bigquery.Client(credentials=credentials,project=PROJECT_NAME)

## 2. Check Tables
---

### List datasets and their tables with their size in GB



In [ ]:
datasets = list(client.list_datasets())

logger1 = logging.getLogger('log')
logger1.setLevel(logging.INFO)
ch = logging.FileHandler("bq-tablesize.log")
ch.setFormatter(logging.Formatter('%(message)s'))
logger1.addHandler(ch)
def logit(content):
	logger1.info(content)


if datasets:
    logger1.info("Datasets in project {}:".format(PROJECT_NAME))
    for dataset in datasets:
        logger1.info("+ \t{}".format(dataset.dataset_id))
        dt = client.get_dataset(dataset.dataset_id)
        tables = list(client.list_tables(dt))
        if tables:
            for table in tables:
                query = """select sum(size_bytes)/1073741824 as size FROM """+ PROJECT_NAME +"."+ dataset.dataset_id +""".__TABLES__ WHERE table_id ='""" + table.table_id + "'"
                query_job = client.query(query)
                for row in query_job:
                  logger1.info("name={}, tablesize={}".format(table.table_id, row["size"]))
        else:
            print("\tThis dataset does not contain any tables.")
else:
    print("{} project does not contain any datasets.".format(PROJECT_NAME))

### Retrieve generated logs

In [ ]:
#download the log file
files.download("bq-tablesize.log")

## 3. Query Performance Test
---

Multi-threaded query executor

**threadnum** defines how many concurrent queries would be executed. Each set of queries is executed the number of thread defines by this number
**test_name** filename where the log will be stored locally
**queries** add your queries here, the query index will be registered in the logs




In [ ]:
import time
import threading
import logging
import random

##### START - To be modified ###
threadnum = 50 # specify the number of concurrent queries (each will execute all queries in the queries list)

logname = "bq-perf-query-test" + str(threadnum) + ".log" # define the name of the local file that will be generated

# indexed and SQL statement, execution order is randomized before being executed
queries =  [
    [1, "SELECT 1"],
    [2, "SELECT 2"],
    [3, "SELECT 3"]
  ]
##### STOP - To be modified ###

# log results
logger = logging.getLogger('log')
logger.setLevel(logging.INFO)
ch = logging.FileHandler(logname)
ch.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(ch)
def logit(content):
	logger.info(content)

# multithreading
exitFlag = 0
threadLock = threading.Lock()
threads = []

# main
def main():
  j = 0
  try:
    for i in range(threadnum):
      t1 = queryThread(i, j)
      threads.append(t1)
      t1.start()
      time.sleep(1)
      j+=1
    for t in threads:
      t.join()
  except:
   		print ("Error: unable to start thread")

#thread executor going through each query
class queryThread (threading.Thread):
	def __init__(self, threadID, counter):
		threading.Thread.__init__(self)
		self.threadID = threadID
		self.counter = counter
	def run(self):
		qr = queries
		random.shuffle(qr)
		i = 0
		for x in qr:
			self.callquery(x, i)
			i+=1

  #job execution and log creation
	def callquery(self, queries, ite):
    start_time = round(time.time() * 1000)
    job_config = bigquery.QueryJobConfig(use_legacy_sql=False, use_query_cache = False, allow_large_results=True)
    query_job = client.query(queries[1], job_config=job_config)
    results = query_job.result()

    job = client.get_job(query_job.job_id, location=REGION)
    stop_time =round(time.time() * 1000)
    delta = stop_time - start_time

    out ="test_" + str(threadnum) +", " + str(self.counter) + ", " + str(ite) + ", " + str(query[0]) +"," + str(start_time) + ", " + str(stop_time) + ", " + str(delta) + ", " + str(job.job_type) + ", " + str(job.state) + ", " + str(job.total_bytes_processed) + ", " + str(job.total_bytes_billed)   + ", " + str(job.slot_millis)
    logit(out)

main()

### Retrieve generated logs

In [ ]:
#download the log file
files.download("bq-perf-query-test" + str(threadnum) + ".log")